In [1]:
# data retrieval for an LLm with RAG
# peft would be used for fine-tuning the model
# https://huggingface.co/docs/peft/index

In [ ]:
# intall dependencies:langchain and openAI
#! pip install -qU langchain openai datasets transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.11.1 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.


Build knowledge base from pdfs
- chunks will be embedded and stored in vector db

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:
text_splitter = RecursiveCharacterTextSplitter( # defines the size of the chunk and overlap so chunks have some info from previous and upcoming chunks -play around with it
    chunk_size = 300, # characters
    chunk_overlap  = 50,# overlap between chunks in characters
    length_function = len, # function to determine the length of a chunk
)

In [ ]:
# make chunks
filepath=".\jroijoifrjs.pdf" # whatever pdf you want
loader = PyPDFLoader(filepath)
chunks = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
# display chunks
for chunk in chunks:
    print("Page_content:\n", chunk.page_content)
    print("Page_Metadata:\n", chunk.metadata)
    print("-------------------------------------------------------------------------------")

In [ ]:
# vector DB:
from langchain.vectorstores import Chroma # open source vector db can be run on local machine or ec2 instance on aws
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [ ]:
# use open source embedding model to embed the chunks
# create the function:
embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") # open AI also has a version of this, but this one can be run on local machine, many available on hugging face

In [ ]:
# explain embeddings:
embedding= embedding_function.embed_docments("This is a test")
print(embedding[0])
print("Dimension of Embedding:",len(embedding[0])) # some give 400-500, openai embedding model gives 1400 dimensions, more dimensions are better for detection accuracy

In [ ]:
# Upload chunks to vector db:
# load it onto chroma
db=Chroma.from_documents(chunks,embedding_function) # ,persist_directory="./chroma_db" # took 13 seconds for him

In [ ]:
print("Chunks in db:",db._collection.count())

In [ ]:
# set up retrieval from vector db
query="what is automimune disease"
retriever=db.as_retriever()
retriever.get_relevant_documents(query) # some chunks that get retrieved, and we will push these into the llm and it will give an answer based on that info


Setting up Rag:

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
# connect to LLM (openai)
from utils import getkey
OPENAI_API_KEY=getkey(key='MYKEY') # get my open ai api key from platform.openai.com

In [ ]:
# set up llm connection
llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    # model_name='gpt-4'
    temperature=0.0,
    max_tokens=100, # could increase
    # streaming=True
)



In [ ]:
# connect vector db to llm:
qa_with_sources=RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    #return_source_documents=True
)

In [ ]:
# rag qa
query="what is automimune disease"
qa_with_sources(query) # this gives the question, answer and source where it is coming from

# when asked something not in the document it will say I don't know how to do that"